Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,Symbol[],[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.13638566100548855, θ=0.8018813834080356)  …  Distributions.Gamma(α=0.14540369181497925, θ=0.920714109129249)  
 Distributions.Gamma(α=0.5683098363905281, θ=0.26458106788743474)     Distributions.Gamma(α=0.44170181692639443, θ=0.3687087748255915) 
 Distributions.Gamma(α=0.7021951551666563, θ=0.1923911896720767)      Distributions.Gamma(α=0.7092153221114026, θ=0.0516499051914967)  
 Distributions.Gamma(α=0.6573255127559685, θ=0.8360241801493695)      Distributions.Gamma(α=0.7048726270375734, θ=0.9490856202855753)  
 Distributions.Gamma(α=0.8360608141005736, θ=0.5636500764361374)      Distributions.Gamma(α=0.02119830958930291, θ=0.02832553971149654))

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.916423  0.930129  0.770073  0.253645
 0.187273  0.628995  0.647929  0.861867
 0.313491  0.416397  0.233424  0.395664)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7761"),Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.0386357  0.0860728  0.296822  0.41982  
 0.955746   0.900698   0.998355  0.701448 
 0.477431   0.386723   0.262832  0.0167867)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,5,3],Dict(:a=>[1,2],:b=>[2,3]),Dict(:a=>[1,2],:b=>[1,2]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,Symbol[],[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.07632574353190402, σ=0.3400599882275994)   …  Distributions.Normal(μ=0.03438855312623201, σ=0.36527619463125327)
 Distributions.Normal(μ=0.3526251808574472, σ=0.8204298579161173)       Distributions.Normal(μ=0.26180156342404115, σ=0.5971632520058308) 
 Distributions.Normal(μ=0.18075051271968312, σ=0.03144904996358644)     Distributions.Normal(μ=0.6031311850584007, σ=0.6851951588100942)  
 Distributions.Normal(μ=0.05146043513964349, σ=0.09653902391947944)     Distributions.Normal(μ=0.16389720892133286, σ=0.07314697019794503)
 Distributions.Normal(μ=0.6675520285212306, σ=0.38028060434646016)      Distributions.Normal(μ=0.7596581036600256, σ=0.5321032890544464)  )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.72423   0.167492  0.416346  0.338683
 0.682916  0.536808  0.652409  0.230596
 0.864777  0.626077  0.996534  0.40275 )

Make a multivariate node

In [8]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]
d[i, j] ~ MvNormal(m, VV)
d.innerinds, d.outerinds

([:i],[:j])

# Make a factor

In [9]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;
value(f)

-34.26473617198157

and a multivariate factor...

In [10]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]

y[i, j] ~ MvNormal(m, VV)
μ[i] ~ Normal(zeros(dims[1]), ones(dims[1]))
Σ[j] ~ Const([diagm(rand(dims[1])) for x in 1:dims[2]])

f = @factor LogMvNormalCanonFactor y μ Σ
value(f)

-28.184517682447847

In [11]:
vars = fieldnames(f)
val_expr = value(typeof(f))
using Base.Cartesian
macroexpand(:(@wrapvars $vars $val_expr (@ntuple 1 i)))

quote  # /Users/jmxp/code/vbgraph/types.jl, line 367:
    δ = E(project(f,:x,(i_1,))) - E(project(f,:μ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 368:
    EΛ = E(project(f,:Λ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 369:
    -(1 / 2) * ((trace(EΛ * ((V(project(f,:x,(i_1,))) .+ V(project(f,:μ,(i_1,)))) .+ δ * δ')) + length(project(f,:x,(i_1,))) * log(2π)) - Elogdet(project(f,:Λ,(i_1,))))
end

In [12]:
f.inds.indices, f.inds.inds_in_factor, f.inds.inds_in_node

([:j],Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]),Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]))

In [13]:
project_inds(f, :x, (5,)), project_inds(f, :μ, (5,))

(Any[5],Any[Colon()])

In [14]:
μ

VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:i],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)])

In [15]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [16]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [17]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1]),Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [18]:
value(obs), value(μ_prior), value(τ_prior)

(-171.53863818342148,-13.593072740907873,0.0449816833123928)

In [19]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,Symbol[],[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,Symbol[],[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,Symbol[],[:i,:j],20x6 Array{Float64,2}:
 0.0923143  0.0358772  0.998653   0.812654    0.234527   0.622584 
 0.261483   0.638221   0.932688   0.63704

In [20]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [21]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [22]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…

In [23]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (8.925646096724032,-11.000000000000002) 
 (11.64606069800854,-11.000000000000002) 
 (12.505773789050734,-11.000000000000002)
 (10.774510890667383,-11.000000000000002)
 (9.367672632241714,-11.000000000000002) 
 (9.478312132275542,-11.000000000000002) 

In [24]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,12.551361349336622)
 (10.0,13.791541155783877)
 (10.0,14.14770258684757) 
 (10.0,13.26105464982084) 
 (10.0,12.500427726428931)
 (10.0,12.558194916805142)

In [25]:
check_conjugate(τ, m)

true

In [26]:
check_conjugate(μ, m)

true

In [27]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [28]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [29]:
update!(m)

In [30]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=-1.4667897115872222, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.4568339006199517, σ=0.7071067811865476)  
 Distributions.Normal(μ=1.0647428571707345, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.15944487829280973, σ=0.7071067811865476)
 Distributions.Normal(μ=-1.1542297508582728, σ=0.7071067811865476) 
 Distributions.Normal(μ=-1.0759958101172635, σ=0.7071067811865476) 

In [31]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=51.96794180583013, θ=0.8999999999999999) 
 Distributions.Gamma(α=17.04186297831486, θ=0.8999999999999999) 
 Distributions.Gamma(α=19.37953672827696, θ=0.8999999999999999) 
 Distributions.Gamma(α=21.07704550343803, θ=0.8999999999999999) 
 Distributions.Gamma(α=41.65238767539627, θ=0.8999999999999999) 
 Distributions.Gamma(α=39.407341242806126, θ=0.8999999999999999)

# A Multivariate Model

We want a simple model of inference for a multivariate normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\pmb{\mu}, \Sigma) \\
    \mu_i \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

In [32]:
d = 5
μ[i] ~ MvNormalCanon(zeros(d), diagm(ones(d)))
Λ[i, i] ~ Wishart(float(d), diagm(ones(d)))
x[i, j] ~ Const(randn(d, 20))
f = @factor LogMvNormalCanonFactor x μ Λ

VB.LogMvNormalCanonFactor{2}(VB.ConstantNode{Float64}(:x,Symbol[],[:i,:j],5x20 Array{Float64,2}:
 -1.51329    -0.0296913  -1.02286   …  -0.437649    0.479336  -1.50745  
 -1.50736     0.0322825   1.17619       0.758332   -0.261801   0.0445372
 -0.0148582   1.11657     0.864689      0.823882    1.21968   -0.996886 
 -0.279171   -0.46222    -0.181406      0.0155962   0.85111   -0.328733 
  0.560821   -0.50859     0.479136      1.17261     0.146444  -0.279035 ),VB.RandomNode{Distributions.MvNormalCanon{P<:PDMats.AbstractPDMat,V<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}}(:μ,[:i],[:scalar],Distributions.MvNormalCanon[FullNormalCanon(
μ: [0.0,0.0,0.0,0.0,0.0]
h: [0.0,0.0,0.0,0.0,0.0]
J: PDMats.PDMat(5,5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0,5x5 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.

In [33]:
naturals(f, μ)

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:


1-element Array{Tuple{Array{Float64,1},Array{Float64,2}},1}:
 ([-43.96907501735852,-5.203230293406979,21.87939115780263,0.9259604655304567,3.2303924983325754],
5x5 Array{Float64,2}:
 -50.0    0.0    0.0    0.0    0.0
   0.0  -50.0    0.0    0.0    0.0
   0.0    0.0  -50.0    0.0    0.0
   0.0    0.0    0.0  -50.0    0.0
   0.0    0.0    0.0    0.0  -50.0)

In [34]:
naturals(f, Λ)

1-element Array{Tuple{Array{Float64,2},Float64},1}:
 (
5x5 Array{Float64,2}:
 22.1248   -2.36063   -1.19156   2.00649  -1.23204 
 -2.36063  20.189      3.42585   2.50784   0.641481
 -1.19156   3.42585   19.3336    1.68971  -0.32105 
  2.00649   2.50784    1.68971  18.981    -2.55833 
 -1.23204   0.641481  -0.32105  -2.55833  15.2965  ,

0.0)

In [35]:
d = 5
μ[i] ~ MvNormal(zeros(d), diagm(ones(d)))
        τ[i] ~ Gamma(1.1 * ones(d), ones(d))
        x[i, j] ~ Const(randn(d, 20))
        f = @factor LogMvNormalCanonFactor x μ τ


VB.LogMvNormalCanonFactor{2}(VB.ConstantNode{Float64}(:x,Symbol[],[:i,:j],5x20 Array{Float64,2}:
  0.763593  0.23016   -0.435491  …   0.745407  -0.328972   0.621371
  0.790728  1.98126   -1.99939       1.74411   -0.28976   -0.23514 
  0.217324  0.117391  -0.453773      0.616475   1.05743    0.471274
 -2.48535   0.883187  -0.201658      0.88273    1.23261    0.293245
 -1.27185   1.36899    0.120966     -1.04567   -0.93001    0.384197),VB.RandomNode{Distributions.MvNormal{Cov<:PDMats.AbstractPDMat,Mean<:Union{Array{Float64,1},Distributions.ZeroVector{Float64}}}}(:μ,[:i],[:scalar],Distributions.MvNormal[FullNormal(
dim: 5
μ: [0.0,0.0,0.0,0.0,0.0]
Σ: 5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0
)
]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:i],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Dist